# SPol data - analysis ready 
- data source : https://data.eol.ucar.edu/dataset/342655 

## Two approaches
data path on GLADE : `/gdex/data/special_projects/ard_radar/`
1. Serving cfradial1 kerchunk reference file (`kerchunk/`)
2. Serving cfradial2 zarr store (`zarr/`)

## Access the analysis ready data
1. UCAR HPC : use the path on GLADE
2. Remote Access on local computer : use the url - `https://data.gdex.ucar.edu/` that replace the GLADE path prefix `/gdex/data/`

## Needed package for data access
- `arm-pyart>=2.1.1`,
- `kerchunk>=0.2.9`
- `xradar>=0.11.0`
- `zarr>=3.1.5`
- `numpy<2.4.0`

## Using `uv` package manager 
simply do 
- `uv sync` (when you are in `radar_arco` folder)
- `source /path/to/radar_arco/.venv/bin/activate`


### Kerchunk access

In [38]:
import xarray as xr

ds_kerchunked = xr.open_dataset('https://data.gdex.ucar.edu/special_projects/ard_radar/kerchunk/cfrad.20220525_020000.545_to_20220525_020048.333_SPOL_PrecipRhi1_RHI.https.json', engine='kerchunk',decode_timedelta=False,chunks={})


In [39]:
# quick view of the kerchunked dataset (lazy loaded - metadata only)
ds_kerchunked

<xarray.Dataset> Size: 167MB
Dimensions:                           (sweep: 2, r_calib: 1, time: 805,
                                       n_points: 1609195, frequency: 1,
                                       range: 1999)
Coordinates:
  * time                              (time) datetime64[ns] 6kB 2022-05-25T02...
  * frequency                         (frequency) float32 4B 2.81e+09
  * range                             (range) float32 8kB 75.0 ... 2.998e+05
    azimuth                           (time) float32 3kB dask.array<chunksize=(805,), meta=np.ndarray>
    elevation                         (time) float32 3kB dask.array<chunksize=(805,), meta=np.ndarray>
Dimensions without coordinates: sweep, r_calib, n_points
Data variables: (12/124)
    volume_number                     float64 8B ...
    platform_type                     object 8B ...
    primary_axis                      object 8B ...
    status_xml                        object 8B ...
    instrument_type                   object 8B ...
    radar_antenna_gain_h              float32 4B ...
    ...                                ...
    SNRVC_F                           (n_points) float32 6MB dask.array<chunksize=(804598,), meta=np.ndarray>
    DBMHC                             (n_points) float32 6MB dask.array<chunksize=(804598,), meta=np.ndarray>
    DBMHC_F                           (n_points) float32 6MB dask.array<chunksize=(804598,), meta=np.ndarray>
    DBMVC                             (n_points) float32 6MB dask.array<chunksize=(804598,), meta=np.ndarray>
    DBMVC_F                           (n_points) float32 6MB dask.array<chunksize=(804598,), meta=np.ndarray>
    CMD_FLAG                          (n_points) float32 6MB dask.array<chunksize=(804598,), meta=np.ndarray>
Attributes: (12/25)
    Conventions:          CF-1.7
    Sub_conventions:      CF-Radial instrument_parameters radar_parameters ra...
    version:              CF-Radial-1.4
    title:                
    institution:          
    references:           
    ...                   ...
    site_name:            Nanliao
    scan_name:            PrecipRhi1
    scan_id:              0
    platform_is_mobile:   false
    n_gates_vary:         true
    ray_times_increase:   true

In [40]:
# load the DBZ variable into memory from the https request
da_dbz = ds_kerchunked['DBZ'].load()

In [41]:
import xradar as xd

# get needed variables to do cfradial1 to cfradial2 conversion
ds_dbz = xr.Dataset()
ds_dbz['DBZ'] = da_dbz
ds_dbz['time'] = ds_kerchunked['time'].load()
ds_dbz['latitude'] = ds_kerchunked['latitude'].load()
ds_dbz['longitude'] = ds_kerchunked['longitude'].load()
ds_dbz['altitude'] = ds_kerchunked['altitude'].load()
ds_dbz['time'] = ds_kerchunked['time'].load()
ds_dbz['sweep_mode'] = ds_kerchunked['sweep_mode'].load()
ds_dbz['sweep_number'] = ds_kerchunked['sweep_number'].load()
ds_dbz['fixed_angle'] = ds_kerchunked['fixed_angle'].load()
ds_dbz['sweep_start_ray_index'] = ds_kerchunked['sweep_start_ray_index'].load()
ds_dbz['sweep_end_ray_index'] = ds_kerchunked['sweep_end_ray_index'].load()

# convert to cfradial2 format
ds_dbz_cf2 = ds_dbz.xradar.to_cf2()


In [42]:
# qucik view of the converted cfradial2 dataset
ds_dbz_cf2

<xarray.DataTree 'root'>
Group: /
│   Dimensions:            (sweep: 2)
│   Dimensions without coordinates: sweep
│   Data variables:
│       latitude           float64 8B nan
│       longitude          float64 8B nan
│       altitude           float64 8B nan
│       sweep_group_name   (sweep) <U9 72B 'sweep_0.0' 'sweep_1.0'
│       sweep_fixed_angle  (sweep) float32 8B 101.0 281.0
├── Group: /sweep_0
│       Dimensions:            (n_points: 1609195, azimuth: 372)
│       Coordinates:
│         * azimuth            (azimuth) float32 1kB 101.0 101.0 101.0 ... 102.9 104.3
│           elevation          (azimuth) float32 1kB -0.5 -0.25 0.0 ... 91.75 92.0 92.25
│           time               (azimuth) datetime64[ns] 3kB 2022-05-25T02:00:00.54500...
│           latitude           float64 8B nan
│           longitude          float64 8B nan
│           altitude           float64 8B nan
│       Dimensions without coordinates: n_points
│       Data variables:
│           DBZ                (n_points) float32 6MB -0.01245 23.67 32.34 ... nan nan
│           sweep_mode         <U3 12B 'rhi'
│           sweep_number       float64 8B 0.0
│           sweep_fixed_angle  float32 4B 101.0
└── Group: /sweep_1
        Dimensions:            (n_points: 1609195, azimuth: 433)
        Coordinates:
          * azimuth            (azimuth) float32 2kB 107.0 109.8 112.6 ... 281.2 281.2
            elevation          (azimuth) float32 2kB 92.33 92.3 92.3 ... 88.75 88.5
            time               (azimuth) datetime64[ns] 3kB 2022-05-25T02:00:16.80900...
            latitude           float64 8B nan
            longitude          float64 8B nan
            altitude           float64 8B nan
        Dimensions without coordinates: n_points
        Data variables:
            DBZ                (n_points) float32 6MB -0.01245 23.67 32.34 ... nan nan
            sweep_mode         <U3 12B 'rhi'
            sweep_number       float64 8B 1.0
            sweep_fixed_angle  float32 4B 281.0

### Zarr store access

In [35]:
import xarray as xr

ds_zarr = xr.open_datatree('https://data.gdex.ucar.edu/special_projects/ard_radar/zarr/cfrad.20220525_015443.665_to_20220525_015709.985_SPOL_PrecipRhi2_RHI.zarr', engine='zarr',chunks={},decode_timedelta=False)

In [36]:
# quick view of the cfradial2 dataset in zarr format
ds_zarr

<xarray.DataTree>
Group: /
│   Dimensions:              (sweep: 11, frequency: 1)
│   Coordinates:
│     * frequency            (frequency) float32 4B 2.81e+09
│   Dimensions without coordinates: sweep
│   Data variables: (12/13)
│       altitude             float64 8B ...
│       altitude_agl         float64 8B ...
│       instrument_type      |S32 32B ...
│       latitude             float64 8B ...
│       longitude            float64 8B ...
│       platform_type        |S32 32B ...
│       ...                   ...
│       status_str           |S10082 10kB ...
│       sweep_fixed_angle    (sweep) float32 44B dask.array<chunksize=(11,), meta=np.ndarray>
│       sweep_group_name     (sweep) object 88B dask.array<chunksize=(11,), meta=np.ndarray>
│       time_coverage_end    |S32 32B ...
│       time_coverage_start  |S32 32B ...
│       volume_number        float64 8B ...
│   Attributes: (12/14)
│       Conventions:         CF-1.7
│       version:             CF-Radial-1.4
│       title:               
│       institution:         
│       references:          
│       source:              
│       ...                  ...
│       instrument_name:     SPOL
│       site_name:           Nanliao
│       scan_name:           PrecipRhi2
│       scan_id:             0
│       platform_is_mobile:  false
│       ray_times_increase:  true
├── Group: /georeferencing_correction
├── Group: /radar_calibration
│       Dimensions:                   ()
│       Data variables: (12/55)
│           antenna_gain_h            float32 4B ...
│           antenna_gain_v            float32 4B ...
│           base_1km_hc               float32 4B ...
│           base_1km_hx               float32 4B ...
│           base_1km_vc               float32 4B ...
│           base_1km_vx               float32 4B ...
│           ...                        ...
│           two_way_radome_loss_v     float32 4B ...
│           two_way_waveguide_loss_h  float32 4B ...
│           two_way_waveguide_loss_v  float32 4B ...
│           xmit_power_h              float32 4B ...
│           xmit_power_v              float32 4B ...
│           zdr_correction            float32 4B ...
├── Group: /radar_parameters
│       Dimensions:                   ()
│       Data variables:
│           radar_antenna_gain_h      float32 4B ...
│           radar_antenna_gain_v      float32 4B ...
│           radar_beam_width_h        float32 4B ...
│           radar_beam_width_v        float32 4B ...
│           radar_receiver_bandwidth  float32 4B ...
...
├── Group: /sweep_7
│       Dimensions:                    (azimuth: 265, range: 1999)
│       Coordinates:
│         * azimuth                    (azimuth) float32 1kB 267.4 268.4 ... 306.2 309.0
│         * range                      (range) float32 8kB 75.0 225.0 ... 2.998e+05
│           altitude                   float64 8B ...
│           elevation                  (azimuth) float32 1kB dask.array<chunksize=(265,), meta=np.ndarray>
│           latitude                   float64 8B ...
│           longitude                  float64 8B ...
│           time                       (azimuth) datetime64[ns] 2kB dask.array<chunksize=(265,), meta=np.ndarray>
│       Data variables: (12/46)
│           antenna_transition         (azimuth) float32 1kB dask.array<chunksize=(265,), meta=np.ndarray>
│           CMD_FLAG                   (azimuth, range) float32 2MB dask.array<chunksize=(67, 1000), meta=np.ndarray>
│           DBMHC                      (azimuth, range) float32 2MB dask.array<chunksize=(67, 1000), meta=np.ndarray>
│           DBMHC_F                    (azimuth, range) float32 2MB dask.array<chunksize=(67, 1000), meta=np.ndarray>
│           DBMVC                      (azimuth, range) float32 2MB dask.array<chunksize=(67, 1000), meta=np.ndarray>
│           DBMVC_F                    (azimuth, range) float32 2MB dask.array<chunksize=(67, 1000), meta=np.ndarray>
│           ...                         ...
│           unambiguous_rang

In [37]:
ds_zarr['sweep_0']['DBZ'].load()

<xarray.DataArray 'DBZ' (azimuth: 252, range: 1999)> Size: 2MB
array([[ -5.271475,  19.810951,  30.773703, ...,        nan,        nan,
               nan],
       [ -5.148946,  23.079596,  30.939476, ...,        nan,        nan,
               nan],
       [ -8.21938 ,  24.978796,  31.105251, ...,        nan,        nan,
          4.590316],
       ...,
       [-17.780254,   9.704103,   9.900511, ...,        nan,        nan,
               nan],
       [-19.403765,   9.192365,  11.477172, ...,  10.159984,  10.205031,
         10.340174],
       [-18.301003,   9.505895,  13.929555, ...,  12.569122,  12.349291,
         12.304243]], shape=(252, 1999), dtype=float32)
Coordinates:
  * azimuth    (azimuth) float32 1kB 90.03 90.03 90.03 ... 90.04 90.04 90.04
  * range      (range) float32 8kB 75.0 225.0 375.0 ... 2.996e+05 2.998e+05
    altitude   float64 8B 10.0
    elevation  (azimuth) float32 1kB -0.5 -0.25 0.0 1.5 ... 61.75 62.0 62.25
    latitude   float64 8B 24.82
    longitude  float64 8B 120.9
    time       (azimuth) datetime64[ns] 2kB 2022-05-25T01:54:43.665000 ... 20...
Attributes:
    long_name:       DBZ
    standard_name:   DBZ
    units:           dBZ
    sampling_ratio:  1.0
    grid_mapping:    grid_mapping